# Resnet 34

In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
# ../input/test-data-1-spec-rainforest/datasets/TrainingSet/spec/a5eaa2ad5/0024_037.png
TEST_PATH='../input/test-data-1-spec-rainforest/datasets/TrainingSet/spec'
sub_csv=pd.read_csv('../input/rfcx-species-audio-detection/sample_submission.csv')
sub_csv_copy=sub_csv.copy()
df_test_data=pd.read_csv('../input/test-data-1-spec-rainforest/test_data_1_list.csv')
sub_csv

In [ ]:
df_test_data

In [ ]:
!pip3 install keras-resnet
!pip install np_utils

In [ ]:
from keras.models import load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

import keras
from keras.utils.np_utils import to_categorical
import keras_resnet.models

shape, classes = (128, 256, 1), 24

x = keras.layers.Input(shape)

model = keras_resnet.models.ResNet34(x, classes=classes)

model.compile("adam", "categorical_crossentropy", ["accuracy"])
#model = load_model('../input/train-rainforest/weights-improvement-18-0.80.hdf5')

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
import numpy as np
# load json and create model
# json_file = open('../input/rainforest-train-py/model.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# loaded_model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("../input/train-rainforest-resnet34/weights-improvement-20-0.79.hdf5")
print("Loaded model from disk")

In [ ]:
def getRandomState():
    RANDOM_SEED = 1
    RANDOM = np.random.RandomState(RANDOM_SEED)
    return RANDOM

In [ ]:
def prepare_data(path):
    x_train=plt.imread(path)
    x_train = x_train.reshape(128, 256, 1) 
    return x_train

In [ ]:
count=0
for audio_id in sub_csv['recording_id']:
    print(count)
    count+=1
    AUDIO_CLASS = os.path.join(TEST_PATH, audio_id)
    #print(AUDIO_CLASS)
    x_onefile_train=[]
    all_audio=os.listdir(AUDIO_CLASS)  
    for one_sec_audio in all_audio:
        one_sec_audio_path=os.path.join(AUDIO_CLASS,one_sec_audio)
        x_train = prepare_data(one_sec_audio_path)
        x_onefile_train.append(x_train)

    x_onefile_train=np.array(x_onefile_train)
    predictions = model.predict(x_onefile_train)
    #print(predictions.shape)
    prob=np.max(predictions, axis=0)
    prob=prob.reshape(1,24)
    #print(prob)
    
    sub = pd.DataFrame(prob, columns=[f"s{i}" for i in range(24)])
    sub["recording_id"] = audio_id
    sub = sub[["recording_id"] + [f"s{i}" for i in range(24)]]
    sub_csv_copy.set_index('recording_id', inplace=True)
    sub_csv_copy.update(sub.set_index('recording_id'))
    sub_csv_copy=sub_csv_copy.reset_index()
#     print(sub)
#     print(audio_id)
#     print(prob)
#     print("---------------------------------------------------------------")
    
    
        #print(one_sec_audio_path)
            

In [ ]:
sub

In [ ]:
sub_csv_copy

In [ ]:
sub_csv_copy.to_csv("submission.csv", index=False)